In [1]:
%pip install lightning

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -U 'wandb>=0.12.10'

Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
import torch.nn as nn
import torchvision.models
import torchvision.transforms as transforms

from datapipeline.asl_image_data_module import ASLImageDataModule
from lightning.pytorch.loggers import WandbLogger
from models.asl_model import ASLModel
from models.training import train, sweep, PROJECT_NAME, ENTITY_NAME
import wandb

In [4]:
img_size = 224

data_transforms = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(5),
    transforms.RandomAffine(0, shear=10, scale=(0.8, 1.2)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
datamodule = ASLImageDataModule(path="/exchange/dspro2/silent-speech/ASL_Pictures_Dataset", transforms=data_transforms)

In [10]:
class ASLCNN_img(nn.Module):
    def __init__(self, kernel_size, dropout=0.2, hidden_dim=128):
        super().__init__()
        self.kernel_size = kernel_size
        self.hidden_dim = hidden_dim
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=self.kernel_size, stride=2, padding=3), # 400 -> 200
            nn.ReLU(),
            nn.MaxPool2d(2),  # 200 -> 100
            
            nn.Conv2d(32, 64, kernel_size=self.kernel_size, padding=1),  # 100
            nn.ReLU(),
            nn.MaxPool2d(2),  # 100 -> 50
            
            nn.Conv2d(64, 128, kernel_size=self.kernel_size, padding=1), # 50
            nn.ReLU(),
            nn.MaxPool2d(2),  # 50 -> 25
        
            nn.Flatten(),
            nn.LazyLinear(self.hidden_dim), #nn.Linear(128 * 25 * 25, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(self.hidden_dim, 28),  # number of classes
        )

    def forward(self, x):
        x = self.model(x)
        return x



In [7]:
sweep_config = {
    "name": "cnn-kernel-lr-dropout-sweep",
    "method": "bayes",  # or "grid", "random"
    "metric": {
        "name": ASLModel.VALID_ACCURACY,
        "goal": "maximize"
    },
    "parameters": {
        "lr": {
            "min": 1e-5,
            "max": 1e-3
        },
        "kernel_size": {
            "values": [3, 5, 7]
        },
        "dropout": {
            "min": 0.2,
            "max": 0.7
        },
        "hidden_dim": {
            "values": [256, 512, 1024],
        },
    }
}


In [11]:
def training_procedure():
    run = wandb.init(project=PROJECT_NAME, entity=ENTITY_NAME)
    config = wandb.config
    run.name = f"ks{run.config.kernel_size}-lr{run.config.lr:.0e}"

    torch_model = ASLCNN_img(kernel_size=config.kernel_size, dropout=config.dropout, hidden_dim=config.hidden_dim)

    model = ASLModel(
        model=torch_model,
        criterion=nn.CrossEntropyLoss(),
        optimizer=torch.optim.Adam(torch_model.parameters(), lr=config.lr),
    )

    logger = WandbLogger(project=PROJECT_NAME, name=f"sweep_cnn_img_{wandb.run.name}")
    train(model, datamodule=datamodule, logger=logger)


In [12]:
sweep(sweep_config=sweep_config, count=5, training_procedure=training_procedure)

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: wddtsc2o
Sweep URL: https://wandb.ai/dspro2-silent-speech/silent-speech/sweeps/wddtsc2o


wandb: Agent Starting Run: z4eiz4im with config:
wandb: 	dropout: 0.5749244774212876
wandb: 	hidden_dim: 256
wandb: 	kernel_size: 3
wandb: 	lr: 0.0009369294303766888
wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.12/site-packages/lightning/pytorch/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type               | Params | Mode 
--------------------------------------------------------------
0 | model          | ASLCNN_img         | 100 K  | train
1 | criterion      | CrossEntropyLoss   | 0      | train
2 | train_accuracy | MulticlassAccuracy | 0      | train
3 | valid_accuracy | MulticlassAccuracy | 0      | train
4 | test_accuracy  | MulticlassAccuracy | 0      | train
--------------------------------------------------------------
100 K     Trainable params
0         Non-trainable params
100 K     Total params
0.402   

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Metric train_accuracy improved by 0.009 >= min_delta = 0.0. New best score: 0.738


Validation: |          | 0/? [00:00<?, ?it/s]

Monitored metric valid_accuracy did not improve in the last 5 records. Best score: 0.036. Signaling Trainer to stop.
wandb: Adding directory to artifact (./silent-speech/8qgzklz8/checkpoints)... Done. 0.1s


epoch,▁▁▁▁▁▁▁▂▂▂▄▄▄▄▅▅▅▅▅▅▅▅▅▅▇▇▇▇▇▇▇▇▇▇██████
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr-Adam-momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr-Adam-weight_decay,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_accuracy_epoch,▁▆█▇██
train_accuracy_step,▇▁█▁█▅▇█▁▁██▃▂██████████▂▁█▁██▆█▆▅████▃▅
train_loss_epoch,█▄▂▂▁▁
train_loss_step,▂▁▁▂▃▂█▂▂▂▁▁▁▁▂▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂
trainer/global_step,▂▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▅▂▂▂▂▃▃▃▃▃▃▃▃▃█▃▃▃▃
valid_accuracy_epoch,▁▁▁▁▁▁
valid_accuracy_step,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁█
